# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa
import binascii
import hashlib


# # Generate a private key
private_key_raw_object = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key raw:", private_key_raw_object.to_string().hex())

private_key_raw = private_key_raw_object.to_string()

# Get the private key
# private_key_hex = private_key_raw.to_string().hex()
# private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_raw, curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key:", private_key.to_string().hex())

# Get the public key
# public_key: ecdsa.VerifyingKey = private_key.get_verifying_key()
public_key = private_key.get_verifying_key()

# Print the public key in hexadecimal format
print(f"Public key: {public_key.to_string("compressed").hex()}")


Private key raw: a94b359bfb90d536715b90c0357e6291850dd6d9f77df229471696bbf3630263
Private key: a94b359bfb90d536715b90c0357e6291850dd6d9f77df229471696bbf3630263
Public key: 0220a625bdfbab8e952ab24ff9f6d7bdb72566f8c7383cc45ecbac75849d0eb02c


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256: bytes = hashlib.sha256(public_key.to_string('compressed')).digest()
print("SHA256: ", sha256.hex())

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()
print("RIPEMD-160: ", hashed_public_key.hex())

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]
print("Address checksum: ", address_checksum.hex())

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum
print("Binary Bitcoin Address: ", binary_bitcoin_address.hex())

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address).decode()
print("Bitcoin address: ", bitcoin_address)

SHA256:  25989afbfc7b7ff042a6b65fe1b8dd4cc7f8cf6d3787fc47aa4fa354a5092230
RIPEMD-160:  ba9ce295187e5d85b00c68399e3b491a44eea433
Address checksum:  e2604c18
Binary Bitcoin Address:  00ba9ce295187e5d85b00c68399e3b491a44eea433e2604c18
Bitcoin address:  1J1idFMPFsXo5f7reucA6adZBA9Qh1hCGf


### Verify a bitcoin address

In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [5]:
def encode_varint(i: int) -> bytes:
    """
    Encode a potentially very large integer into varint bytes. The length should be
    specified in little-endian.

    https://bitcoin.org/en/developer-reference#compactsize-unsigned-integers
    """
    if i < 253:
        return bytes([i])
    elif i < 0x10000:
        return b"\xfd" + i.to_bytes(2, "little")
    elif i < 0x100000000:
        return b"\xfe" + i.to_bytes(4, "little")
    elif i < 0x10000000000000000:
        return b"\xff" + i.to_bytes(8, "little")
    else:
        raise ValueError("Integer is too large: %d" % i)

In [6]:
# Path: concepts_bitcoin_transactions.ipynb
import base64

private_key_raw_1 = private_key_raw_object
msg = b"Hello world!"
msg_size = encode_varint(len(msg))
message_hash = hashlib.sha256(hashlib.sha256(b'\x18Bitcoin Signed Message:\n' + msg_size + msg).digest()).digest()
print("Message hash:", message_hash.hex())

signature_ = private_key_raw_1.sign_deterministic(message_hash)
# Use the private key to sign a message
# message = bytes("Bitcoin Signed Message:\nHello world!", "utf-8")
# message = b"\x18Bitcoin Signed Message:\n" + msg_size + msg
# message_size = encode_varint(len(message))
# signature_ = private_key.sign_deterministic(hashlib.sha256(hashlib.sha256(message).digest()).digest())
# print("Signature:", signature_.hex())
print(f'Signature in b64: {base64.b64encode(signature_).hex()}')
print("Signature hex:", binascii.hexlify(signature_).decode('ascii'))

# Verify the signature
print(public_key.verify(signature_, message_hash))


Message hash: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature in b64: 744877795a58524f4d6c6b2b2f6e43506a646961696c554c657167505a6d6f6361453350635262636170793938546c456b50533032547865706d4a54436272592b746b6b47304e3053317972426148324a67725133413d3d
Signature hex: b47c3265744e32593efe708f8dd89a8a550b7aa80f666a1c684dcf7116dc6a9cbdf1394490f4b4d93c5ea6625309bad8fad9241b43744b5cab05a1f6260ad0dc
True


In [7]:
# Get the private key
# private_key_hex = 'your_private_key_here'
# private_key_bytes = bytes.fromhex(private_key_hex)
# private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

# The message to sign
# message = 'your_message_here'

# Sign the message
signature = private_key_raw_1.sign(message_hash)
print(f"Signature: {signature.hex()}")

# Print the signature in base64 format
signature_b64 = base64.b64encode(signature).hex()
print(f"Signature b64: {signature_b64}")

# Get the public key
public_key: ecdsa.VerifyingKey = private_key_raw_1.get_verifying_key()
print(f"Public key: {public_key.to_string('compressed').hex()}")


Signature: 68b07eb4809ff64c8b0dbc5191beed756e19ca907f0cc8b057e742794624cce24f1da289646d58948731f4c32207c7e6f7cfb3a8a3c389c05eb8a93e504ca98a
Signature b64: 614c422b74494366396b794c446278526b6237746457345a7970422f444d6977562b64436555596b7a4f4a5048614b4a5a4731596c496378394d4d694238666d39382b7a714b504469634265754b6b2b5545797069673d3d
Public key: 0220a625bdfbab8e952ab24ff9f6d7bdb72566f8c7383cc45ecbac75849d0eb02c


### Verify a message

In [8]:
# Use the public key to verify a message
# Verify the signature
try:
    public_key.verify(signature, message_hash)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Signature valid


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [9]:
# pip install pybitcoin

In [10]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [11]:
# pip install bitcoin-utils

In [12]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [13]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [14]:
# pip install python-bitcoinlib

In [15]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress, CBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_)
# bitcoin_address_ = CBitcoinAddress(bitcoin_address_)
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_msg.magic} {bitcoin_msg.message} {bitcoin_msg.GetHash().hex()}")

signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_}")

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: KzVbNeCLTjRncqYKpNh6rsaJeWCiCvJg7RB54PTWUMAnnv353vAu
Public key: 02f9c61b27bf511bb0daeef305c23381df8a75fb8e5b169d3796e63e40bc10810e
Bitcoin address: 19U5dCcukd1nYfn5UWtyELt7zQKkj98mm4
Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
b'\x88\xfc\xcc\x03\x97f\x98\x93\x96\x15\xd4\x9cg\xf9\xe1F\xfb)\x07S\n\xf6\x8d\x88\xe1\x18\x01\xbc\x17\xa4\x04\xe3\x1d\xfb\x8e\xba\x96\xff\xb0k\x89\xaat\xf3H\xe9\x80\xa2\x9c\x91t\xe7\x9e\x8a\x07\x81\xe3\x03{\xdc\x175\x0e\x91' 1
Signature: b'IIj8zAOXZpiTlhXUnGf54Ub7KQdTCvaNiOEYAbwXpATjHfuOupb/sGuJqnTzSOmAopyRdOeeigeB4wN73Bc1DpE='
Signature valid? True


In [16]:
# verify the handmade message from the top
# print(f'Private Key BAse58 Encoded: {base58.b58encode(private_key_raw + b"\x01")}')
print(f'Private key not encoded: {private_key_raw.hex() + "01"}')
# bitccoin_private_key = CBitcoinSecret.from_secret_bytes(private_key_raw)
bitccoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_raw.hex() + "01"), 0)
bitccoin_private_key_bytes = bitccoin_private_key

# print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()[:-2]}")
print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()}")

# bitcoin_public_key_bytes = bitccoin_private_key_bytes.pub
# print(f'Public key base58 Encoded: {base58.b58encode(private_key.get_verifying_key().to_string('compressed'))}')
print(f'bitcoin public not encoded: {private_key.get_verifying_key().to_string('compressed').hex()}')

def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
# bitcoin_public_key_bytes = compress_public_key(private_key_raw.get_verifying_key().to_string())
bitcoin_public_key_bytes = private_key.get_verifying_key().to_string('compressed')
print(f"Bitcoin public key: {bitcoin_public_key_bytes.hex()}")

print(f"Bitcoin address from top: {bitcoin_address}")
bitcoin_address_bytes = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(bitcoin_public_key_bytes.hex()))
print(f"Bitcoin address from pubkey: {bitcoin_address_bytes}")

# bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(bytes.fromhex(bitcoin_public_key_bytes.hex()), 0)
# print(f"Bitcoin address: {bitcoin_address_bytes}")


Private key not encoded: a94b359bfb90d536715b90c0357e6291850dd6d9f77df229471696bbf363026301
Bitcoin private key: a94b359bfb90d536715b90c0357e6291850dd6d9f77df229471696bbf363026301
bitcoin public not encoded: 0220a625bdfbab8e952ab24ff9f6d7bdb72566f8c7383cc45ecbac75849d0eb02c
Bitcoin public key: 0220a625bdfbab8e952ab24ff9f6d7bdb72566f8c7383cc45ecbac75849d0eb02c
Bitcoin address from top: 1J1idFMPFsXo5f7reucA6adZBA9Qh1hCGf
Bitcoin address from pubkey: 1J1idFMPFsXo5f7reucA6adZBA9Qh1hCGf


In [17]:

bitcoin_message_bytes = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_message_bytes.magic} {bitcoin_message_bytes.message} {bitcoin_message_bytes.GetHash().hex()}")

print(f'signature from top: {signature_.decode()}')
# print(f'signature from top b64: {base64.b64encode(signature)}')
# signature_bytes = signature.__bytes__()
signature_bytes_b64 = base64.b64encode(signature_)
print(f"Signature from top b64: {signature_bytes_b64}")

#sign message using bitcoin library
signature_bytes = SignMessage(bitccoin_private_key, bitcoin_message_bytes)
print(f"Signature bytes: {signature_bytes.decode()}")

# print(f'signature_bytes_hex: {base64.b64encode(signature_bytes).hex()}')
print(f'signature bytes decoded: {base64.b64decode(signature_bytes).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_bytes, bitcoin_message_bytes, signature_bytes)}")

Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
signature from top: IIj8zAOXZpiTlhXUnGf54Ub7KQdTCvaNiOEYAbwXpATjHfuOupb/sGuJqnTzSOmAopyRdOeeigeB4wN73Bc1DpE=
Signature from top b64: b'SUlqOHpBT1hacGlUbGhYVW5HZjU0VWI3S1FkVEN2YU5pT0VZQWJ3WHBBVGpIZnVPdXBiL3NHdUpxblR6U09tQW9weVJkT2VlaWdlQjR3TjczQmMxRHBFPQ=='
b"OW\xd0\xff\x9f\xe6g\x88>\x143\x92[\xf7\x9d\x8e%\x8b\x01\xd1xb\x89\xae\x0c%c\xd2\xc50\xc5\x80\x0f,\x9b\xa07F'\xd4\xb6_{\\?3Q\xd2\x81\x9b\x11\xa0r\xe3\xb3\xc2>\xdd\xdc*?\x8e\x86\x00" 1
Signature bytes: IE9X0P+f5meIPhQzklv3nY4liwHReGKJrgwlY9LFMMWADyyboDdGJ9S2X3tcPzNR0oGbEaBy47PCPt3cKj+OhgA=
signature bytes decoded: 204f57d0ff9fe667883e1433925bf79d8e258b01d1786289ae0c2563d2c530c5800f2c9ba0374627d4b65f7b5c3f3351d2819b11a072e3b3c23edddc2a3f8e8600
Signature valid? True


In [20]:
# Get the private key
private_key_hex = private_key_raw.hex()
private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)
print(f"Private key: {private_key.to_string().hex()}")

# Get the public key
public_key_hex = private_key.get_verifying_key().to_string('compressed').hex()
public_key_bytes = binascii.unhexlify(public_key_hex)
public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)
print(f"Public key: {public_key.to_string('compressed').hex()}")

# The byte string to sign
message = b'Hello world!'

# Sign the message
signature = private_key.sign(message_hash)

# Print the signature in hexadecimal format
print(f"Signature: {binascii.hexlify(signature).decode()}")

signature_hex = binascii.hexlify(signature).decode()

# The signature
signature_unhex = binascii.unhexlify(signature_hex.encode('ascii'))
print(f"Signature unhex: {signature_unhex}")

# Verify the signature
try:
    public_key.verify(signature, message_hash)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Private key: a94b359bfb90d536715b90c0357e6291850dd6d9f77df229471696bbf3630263
Public key: 0220a625bdfbab8e952ab24ff9f6d7bdb72566f8c7383cc45ecbac75849d0eb02c
Signature: a9cc5ac6361434a586a8d494622584fbcf39de7999ea05d536fe5a24a2527a1808b28fbff6cc47e4371d21a324bef6ff56967d8afee617a1c15d8716c952cc4e
Signature unhex: b'\xa9\xccZ\xc66\x144\xa5\x86\xa8\xd4\x94b%\x84\xfb\xcf9\xdey\x99\xea\x05\xd56\xfeZ$\xa2Rz\x18\x08\xb2\x8f\xbf\xf6\xccG\xe47\x1d!\xa3$\xbe\xf6\xffV\x96}\x8a\xfe\xe6\x17\xa1\xc1]\x87\x16\xc9R\xccN'
Signature valid
